In [15]:
import json
import pandas as pd

In [16]:
context = json.load(open('../Data/merged_data_file.json'))
data = []
for key, value in context.items():
    temp = {
        'ID': key,
        'Context': value['context']
        }
    data.append(temp)
context = pd.DataFrame(data)
context['ID'] = context['ID'].astype(int)

In [17]:
jd_data = json.load(open('../Data/final_data_new.json'))
df = pd.json_normalize(jd_data)

df.rename(columns = {'_id':'ID','claim':'Claim', 'metadata.context_id':'Context_ID', 'metadata.label':'Verdict', 'metadata.evidence':'Evidence'}, inplace = True)

In [18]:
df['Context_ID'] = df['Context_ID'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948 entries, 0 to 12947
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          12948 non-null  object
 1   Claim       12948 non-null  object
 2   Context_ID  12948 non-null  int64 
 3   Verdict     12948 non-null  object
 4   Evidence    12948 non-null  object
dtypes: int64(1), object(4)
memory usage: 505.9+ KB


In [19]:
context_df = pd.merge(df, context, left_on='Context_ID', right_on='ID', how='inner')
context_df.drop(columns='Context_ID', inplace=True)
context_df.rename(columns={'ID_x': 'Claim_ID', 'ID_y': 'Context_ID','Label': 'Verdict'}, inplace=True)
df = context_df
df.head()

,Claim_ID,Claim,Verdict,Evidence,Context_ID,Context
0,0,Nữ du khách người Nga bị voi tấn công và gãy c...,HỖ TRỢ,"Nữ du khách người Nga đã bị gãy chân, người cò...",10000,"Nữ du khách bị voi tóm lấy, thả xuống đất khi ..."
1,1,Sự việc xảy ra khi nạn nhân đang chuẩn bị trèo...,HỖ TRỢ,Sự việc xảy ra trong lúc nạn nhân đang chờ để ...,10000,"Nữ du khách bị voi tóm lấy, thả xuống đất khi ..."
2,2,Nữ du khách người Nga không bị gãy chân mà chỉ...,PHỦ NHẬN,"Nữ du khách người Nga đã bị gãy chân, người cò...",10000,"Nữ du khách bị voi tóm lấy, thả xuống đất khi ..."
3,3,Sự việc không xảy ra khi nạn nhân đang chuẩn b...,PHỦ NHẬN,Sự việc xảy ra trong lúc nạn nhân đang chờ để ...,10000,"Nữ du khách bị voi tóm lấy, thả xuống đất khi ..."
4,4,Cửa hàng đã tạm thời đóng cửa từ ngày 8/5 đến ...,HỖ TRỢ,"Trong bài viết, chủ quán cho hay sẽ tạm thời d...",10001,Vụ dòi bò lúc nhúc trong miếng pate: Quán tạm ...


# SBERT

In [20]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
list_context = context_df['Context'].tolist()
list_claim = context_df['Claim'].tolist()
ID = context_df['Claim_ID'].tolist()
model = SentenceTransformer("hiieu/halong_embedding")

In [ ]:
evidence = []

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

text_splitter = RecursiveCharacterTextSplitter(
    separators=['. ', ', '],
    chunk_size=256,
    chunk_overlap=70,
)
threshold = 0.85
start = time.time()

for i in range(len(list_context)):
  raw_text = list_context[i]
  query = list_claim[i]
  chunks = text_splitter.split_text(raw_text)

  embeddings = model.encode(chunks)

  query_embedding = model.encode([query])
  similarities = [cosine_similarity(query_embedding, embedding) for embedding in embeddings]

  best_match_idx = np.argmax(similarities)
  best_match_chunk = chunks[best_match_idx]

  evidence.append(best_match_chunk)

df_kq = pd.DataFrame({'ID_Claim': ID, 'evidence': evidence})
df_kq.to_csv('Final_210_70.csv', index=False)
display(df_kq.head())
display(df_kq.verdict.value_counts())

In [ ]:
verdicts = []
evidences = []

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

text_splitter = RecursiveCharacterTextSplitter(
    separators=['. ', ', '],
    chunk_size=256,
    chunk_overlap=100,
)

threshold = 0.85

for i in range(len(list_context)):
    raw_text = list_context[i]
    query = list_claim[i]
    chunks = text_splitter.split_text(raw_text)

    embeddings = model.encode(chunks)

    query_embedding = model.encode([query])
    similarities = [cosine_similarity(query_embedding, embedding) for embedding in embeddings]

    if isinstance(chunks, list):
        top_k_indices = np.argsort(similarities)[-5:][::-1]  # Get indices of top 5 similarities in descending order
        top_k_chunks = [chunks[int(idx)] for idx in top_k_indices]
        top_k_similarities = [similarities[int(idx)] for idx in top_k_indices]
        top_k_verdicts = [0 if similarity > threshold else 1 for similarity in top_k_similarities]

        verdicts.append(top_k_verdicts)
        evidences.append(top_k_chunks)
    else:
        pass

default_value = None  # You can change this to any default value you want

df_kq = pd.DataFrame({
    'ID_Claim': ID,
    'verdict_1': [v[0] if len(v) > 0 else default_value for v in verdicts],
    'verdict_2': [v[1] if len(v) > 1 else default_value for v in verdicts],
    'verdict_3': [v[2] if len(v) > 2 else default_value for v in verdicts],
    'verdict_4': [v[3] if len(v) > 3 else default_value for v in verdicts],
    'verdict_5': [v[4] if len(v) > 4 else default_value for v in verdicts],
    'evidence_1': [e[0] if len(e) > 0 else "" for e in evidences],
    'evidence_2': [e[1] if len(e) > 1 else "" for e in evidences],
    'evidence_3': [e[2] if len(e) > 2 else "" for e in evidences],
    'evidence_4': [e[3] if len(e) > 3 else "" for e in evidences],
    'evidence_5': [e[4] if len(e) > 4 else "" for e in evidences]
})
df_kq.to_csv('Final_256_100.csv', index=False)

# BM25

In [ ]:
import pandas as pd
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize, sent_tokenize

# Hàm để tìm evidence trong context cho mỗi claim
def find_evidence(claim, context):
    # Tách context thành các câu
    context_sentences = sent_tokenize(context)
    
    # Tokenize các câu trong context và claim
    tokenized_context = [word_tokenize(sentence) for sentence in context_sentences]
    tokenized_claim = word_tokenize(claim)
    
    # Tạo mô hình BM25
    bm25 = BM25Okapi(tokenized_context)
    
    # Tìm các đoạn liên quan nhất cho claim
    scores = bm25.get_scores(tokenized_claim)
    ranked_sentences = sorted(zip(scores, context_sentences), reverse=True)
    
    # Lấy câu có điểm số cao nhất
    best_sentence = ranked_sentences[0][1] if ranked_sentences else ""
    
    return best_sentence

# Áp dụng hàm tìm evidence cho mỗi dòng trong DataFrame
df['Evidence_Predict'] = df.apply(lambda row: find_evidence(row['Claim'], row['Context']), axis=1)

# Lưu kết quả vào tệp CSV mới
df.to_csv('output_with_evidence.csv', index=False)

print("Đã tìm kiếm evidence và lưu kết quả vào tệp 'output_with_evidence.csv'.")

In [ ]:
import pandas as pd
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize, sent_tokenize

# Hàm để tìm top 5 evidence trong context cho mỗi claim
def find_top_5_evidence(claim, context):
    # Tách context thành các câu
    context_sentences = sent_tokenize(context)
    
    # Tokenize các câu trong context và claim
    tokenized_context = [word_tokenize(sentence) for sentence in context_sentences]
    tokenized_claim = word_tokenize(claim)
    
    # Tạo mô hình BM25
    bm25 = BM25Okapi(tokenized_context)
    
    # Tìm các đoạn liên quan nhất cho claim
    scores = bm25.get_scores(tokenized_claim)
    ranked_sentences = sorted(zip(scores, context_sentences), reverse=True)
    
    # Lấy top 5 câu có điểm số cao nhất
    top_5_sentences = [sentence for _, sentence in ranked_sentences[:5]]
    
    # Nếu ít hơn 5 câu, điền thêm bằng chuỗi rỗng
    while len(top_5_sentences) < 5:
        top_5_sentences.append("")
    
    return top_5_sentences

# Áp dụng hàm tìm top 5 evidence cho mỗi dòng trong DataFrame
df[['Evidence_Predict_1', 'Evidence_Predict_2', 'Evidence_Predict_3', 'Evidence_Predict_4', 'Evidence_Predict_5']] = df.apply(
    lambda row: pd.Series(find_top_5_evidence(row['Claim'], row['Context'])), axis=1)

# Lưu kết quả vào tệp CSV mới
df.to_csv('output_with_top5_evidence.csv', index=False)

print("Đã tìm kiếm top 5 evidence và lưu kết quả vào tệp 'output_with_top5_evidence.csv'.")